# Setup

In [30]:
import argparse

In [31]:
import numpy as np
import os
import glob
import pandas as pd
import sys

In [3]:
import yaml

In [36]:
import os

In [29]:
# # Retrieve config_fp if passed via command line
# parser = argparse.ArgumentParser()
# parser.add_argument(
#     '--config_fp',
#     type = str,
#     help = 'Location of your config file.',
#     default = ,
# )
# cli_args, unknown_args = parser.parse_known_args()

In [34]:
with open( './config.yml', "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

## Process Config

In [32]:
print( os.getcwd() )

/Users/zhafen/repos/press-dashboard/press_dashboard_library


In [10]:
data_fp = os.path.join( config['data_dir'], config['input_dirname'], config['website_data_file_pattern'] )
data_fp = glob.glob( data_fp )[0]

In [11]:
press_office_data_fp = os.path.join( config['data_dir'], config['input_dirname'], config['press_office_data_file_pattern'] )
press_office_data_fp = glob.glob( press_office_data_fp )[0]

In [12]:
output_dir = os.path.join( config['data_dir'], config['output_dirname'] )
os.makedirs( output_dir, exist_ok=True )

In [13]:
grouping_labels = [ group_by_i.lower().replace( ' ', '_' ) for group_by_i in config['groupings'] ]

In [14]:
data_fp = os.path.join( config['data_dir'], config['input_dirname'], config['website_data_file_pattern'] )
data_fp = glob.glob( data_fp )[0]

In [15]:
press_office_data_fp = os.path.join( config['data_dir'], config['input_dirname'], config['press_office_data_file_pattern'] )
press_office_data_fp = glob.glob( press_office_data_fp )[0]

In [16]:
output_dir = os.path.join( config['data_dir'], config['output_dirname'] )
os.makedirs( output_dir, exist_ok=True )

In [17]:
grouping_labels = [ group_by_i.lower().replace( ' ', '_' ) for group_by_i in config['groupings'] ]

# Transform Website Data

## Preprocessing

In [18]:
# Load data
df = pd.read_csv( data_fp, parse_dates=[ 'Date', ] )

In [19]:
# Drop drafts
df.drop( df.index[df['Date'].dt.year == 1970], axis='rows', inplace=True )

In [20]:
# Drop weird articles---ancient ones w/o a title or press type
df.dropna( axis='rows', how='any', subset=[ 'Title', 'Press Types', ], inplace=True )

In [21]:
# Get rid of HTML ampersands
for str_column in [ 'Title', 'Research Topics', 'Categories' ]:
    df[str_column] = df[str_column].str.replace( '&amp;', '&' )

In [22]:
# Get date bins
start_year = df['Date'].min().year - 1
end_year = df['Date'].max().year + 1
date_bins = pd.date_range(
    '{} {}'.format( config['year_start'], start_year ),
    pd.Timestamp.now() + pd.offsets.DateOffset( years=1 ),
    freq = pd.offsets.DateOffset( years=1 ),
)
date_bin_labels = date_bins.year[:-1]

In [23]:
# Add the year published (using the above start date )
df['Year'] = pd.cut( df['Date'], date_bins, labels=date_bin_labels ) 

## Grouped Counts

In [24]:
for i, group_by_i in enumerate( config['groupings'] ):
    df_i = df.copy()

    # Explode and group
    df_i[group_by_i] = df_i[group_by_i].str.split( '|' )
    df_i = df_i.explode( group_by_i )

    # Get counts
    counts = df_i.pivot_table( index='Year', columns=group_by_i, values='id', aggfunc='count' )
    
    # Save
    output_fn = 'counts.{}.csv'.format( grouping_labels[i] )
    output_fp = os.path.join( output_dir, output_fn )
    counts.to_csv( output_fp, )

# Coordinate Manual Data